<a href="https://colab.research.google.com/github/Arzeus17/challenge/blob/main/Reporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CHALLENGE MERCADO LIBRE

## Primera Parte - SQL

### Objetivo

El objetivo de esta parte del desafío es diseñar un Diagrama de Entidad-Relación (DER) que responda al modelo del negocio y responder mediante SQL a diversas preguntas planteadas.

## Diseño del DER

Aquí se muestra el Diagrama de Entidad-Relación (DER) diseñado en MySQL Workbench:

![DER](https://github.com/Arzeus17/challenge/blob/main/img/DER.jpg)

### Generación del Script DDL

El siguiente paso consistió en generar el script DDL para la creación de las tablas del DER. Este script se creó utilizando Visual Studio Code y la extensión de MySQL. El script resultante se ha guardado con el nombre "create_tables.sql" y se presenta a continuación:

```sql
--script DDL generado
CREATE TABLE Customer (
    Customer_id INT PRIMARY KEY,
    Email VARCHAR(255),
    Nombre VARCHAR(50),
    Apellido VARCHAR(50),
    Sexo CHAR(1),
    Direccion VARCHAR(255),
    Fecha_Nacimiento DATE,
    Telefono VARCHAR(20)
);

CREATE TABLE Orders (
    Order_id INT PRIMARY KEY,
    Customer_id INT,
    Fecha_Compra DATE,
    FOREIGN KEY (Customer_id) REFERENCES Customer(Customer_id)
);

CREATE TABLE Item (
    Item_id INT PRIMARY KEY,
    nombre VARCHAR(100),
    Descripcion VARCHAR(255),
    Precio DECIMAL(10, 2),
    Estado VARCHAR(20),
    Fecha_Baja DATE,
    Categoria_id INT,
    FOREIGN KEY (Categoria_id) REFERENCES Category(Categoria_id)
);

CREATE TABLE Category (
    Categoria_id INT PRIMARY KEY,
    nombre VARCHAR(50),
    path VARCHAR(50)
);
```

### Generación del Código SQL para Respuestas de Negocio

Después de diseñar el DER y crear las tablas necesarias en el DDL, el siguiente paso fue generar el código SQL para responder a las situaciones planteadas en el desafío. Este código se guardó en un archivo con el nombre "respuestas_negocio.sql". A continuación, se detallan los pasos seguidos:

1. **Listar los usuarios que cumplen años el día de hoy y cuya cantidad de ventas realizadas en enero de 2020 sea superior a 1500:**

    Se utilizó una consulta SQL que filtra los usuarios que cumplen años en la fecha actual y que hayan realizado más de 1500 ventas en enero de 2020.

```sql
--script generado
SELECT c.Nombre, c.Apellido
FROM Customer c
INNER JOIN Orders o ON c.Customer_id = o.Customer_id
--Filtro los clientes que que cumplen años hoy
WHERE MONTH(c.Fecha_Nacimiento) = MONTH(NOW()) AND DAY(c.Fecha_Nacimiento) = DAY(NOW())
--Filtro las ordenes realizadas en enero de 2020
AND o.Fecha_Compra BETWEEN '2020-01-01' AND '2020-01-31'
GROUP BY c.Customer_id
--Filtro solo los clientes que tienen mas de 1500 ordenes en enero de 2020
HAVING COUNT(o.order_id) > 1500;
```

2. **Por cada mes del 2020, obtener el top 5 de usuarios que más vendieron en la categoría Celulares:**

    Se elaboró una consulta SQL que calcula el top 5 de usuarios que más vendieron en la categoría de Celulares para cada mes del año 2020.

```sql
--script generado
--Extraigo por años y mes de compra, y nombre y apellido del comprador
SELECT
    year,
    month,
    Nombre,
    Apellido,
--Estimo la cantidad de ventas, productos y el total transaccionado por vendedor
    cantidad_ventas,
    cantidad_productos,
    monto_total_transaccion
FROM
    (
       --Subconsulto para tener el top 5 de usuarios por mes
       SELECT
            YEAR(o.Fecha_Compra) AS year,
            MONTH(o.Fecha_Compra) AS month,
            c.Nombre AS Nombre,
            c.Apellido AS Apellido,
            COUNT(o.Order_id) AS cantidad_ventas,
            SUM(o.Cantidad) AS cantidad_productos,
            SUM(o.Monto_Total) AS monto_total_transaccion,
            ROW_NUMBER() OVER (PARTITION BY YEAR(o.Fecha_Compra), MONTH(Fecha_Compra), o.Customer_id ORDER BY SUM(Monto_Total) DESC) AS ranking
        FROM
            Orders o
                INNER JOIN Customer c ON o.Customer_id = c.Customer_id
                INNER JOIN Item i ON o.Item_id = i.Item_id
                INNER JOIN Category cat ON i.Categoria_id = cat.Categoria_id
        --Filtro por la categoria 'Celulares'
        WHERE
            cat.nombre ='Celulares'
            AND o.Fecha_Compra BETWEEN '2020-01-01' AND '2020-12-31'
            GROUP BY
                YEAR(o.Fecha_Compra), MONTH(o.Fecha_Compra), c.Customer_id
    ) AS top_users_per_month
WHERE
    ranking <= 5 --top 5
ORDER BY
    year, month, monto_total_transaccion;
```

3. **Poblar una nueva tabla con el precio y estado de los ítems al final del día:**

    Se ejecutó una sentencia SQL para crear una nueva tabla llamada "Estado_Items" que contiene el precio y estado de los ítems al final del día. Esta tabla se diseñó para ser reprocesable y conservar únicamente el último estado informado por la clave primaria definida.

```sql
--script generado
DELIMITER //

CREATE PROCEDURE UpdateOrInsertItemPriceAndState()
BEGIN
    --Elimino datos anteriores de la tabla de precios y estados de los items
    TRUNCATE TABLE ItemPriceAndState;

    --Inserto nuevos datos en la tabla de precios y estados de los datos
    INSERT INTO ItemPriceAndState (Item_id, Precio, Estado, Fecha)
    SELECT i.Item_id, i.Precio, i.Estado, CURDATE() AS Fecha
    FROM Item i
    WHERE i.Fecha_Baja IS NULL; --considero solo los items activos

    --Actualizo registros existentes en la tabla Item
    UPDATE Item i
    JOIN (
        --Subconsulto para identificar el estado mas reciente para cada item
        SELECT Item_id, MAX(Fecha_Baja) AS UltimaFecha
        FROM Item
        GROUP BY Item_id
    ) AS UltimosEstados ON Item_id = UltimosEstados.Item_id
    SET i.Precio = (SELECT Precio FROM Item WHERE Item_id = i.Item_id AND Fecha_Baja = UltimosEstados.UltimaFecha),
        i.Estado = (SELECT Estado FROM Item WHERE Item_id = i.Item_id AND Fecha_Baja = UltimosEstados.UltimaFecha)
    WHERE i.Fecha_Baja = UltimosEstados.UltimaFecha;

    --Inserto nuevos registros en la tabla Item
    INSERT INTO Item (Item_id, Precio, Estado, Fecha_Baja)
    SELECT ips.Item_id, ips.Precio, ips.Estado, ips.Fecha
    FROM ItemPriceAndState ips
    WHERE NOT EXISTS (
        SELECT 1
        FROM Item WHERE Item.Item_id = ips.Item_id
    );
END//

DELIMITER ;
```

Estos pasos fueron realizados utilizando un entorno de desarrollo, específicamente Visual Studio Code con la extensión MySQL. Finalmente, el código SQL generado se guardó en un archivo llamado "respuestas_negocio.sql" para su posterior referencia.

## Segunda Parte - APIs (Deseable)

### Objetivo
El objetivo de esta parte es realizar un análisis sobre la oferta de productos en el sitio MercadoLibre.com.ar, centrándonos en cinco marcas de teléfonos celulares seleccionadas para evaluar.

### Consignas
1. **Barrido de lista de ítems:** Utilicé el servicio público de MercadoLibre para obtener una lista de más de 150 ítems. En este caso, elegi cinco marcas de celulares, como ejemplos de dispositivos portátiles:iPhone, Samsung Galaxy, Google Pixel, Xiaomi Redmi y Huawei P30.
2. **Obtención de detalles de cada ítem:** Por cada ítem obtenido en el paso anterior, se realizó una solicitud GET al recurso público correspondiente al Item_Id para obtener más detalles sobre el producto.
3. **Almacenamiento de resultados:** Escribi los resultados en un archivo plano delimitado por comas, desnormalizando el JSON obtenido en el paso anterior. En este archivo, almacé tantos campos como sea necesario para capturar las variables relevantes a analizar.

### Implementación
El consumo de la API de MercadoLibre se llevó a cabo utilizando Visual Studio Code y la extensión de Python. A continuación, se presenta el código generado:

```python
import requests
import csv

# Enlisto los términos de búsqueda
search_terms = ["iPhone", "Samsung Galaxy", "Google Pixel", "Xiaomi Redmi", "Huawei P30"]

# Inicializo lista para almacenar todos los ítems de todas las búsquedas
all_items = []

# Itero sobre cada término de búsqueda
for search_term in search_terms:
    # Realizo solicitud GET a la API de Mercado Libre para el término de búsqueda actual
    response = requests.get(f"https://api.mercadolibre.com/sites/MLA/search?q={search_term}&limit=50")
    data = response.json()

    # Proceso cada resultado de la búsqueda actual
    for result in data.get('results', []):
        item_id = result.get('id')
        if item_id:
            # Realizo solicitud GET para obtener detalles del ítem actual
            item_response = requests.get(f"https://api.mercadolibre.com/items/{item_id}")
            item_data = item_response.json()
            
            # Selecciono campos de interés y agregarlos a la lista de ítems
            item_info = {
                'Search Term': search_term,
                'Title': item_data.get('title'),
                'Price': item_data.get('price'),
                'Condition': item_data.get('condition'),
                'Sold Quantity': item_data.get('sold_quantity'),
                'Location': item_data.get('seller_address', {}).get('city', ''),
                'Discount': item_data.get('discount'),
                'Seller Rating': item_data.get('seller_rating'),
                'Seller Sales Completed': item_data.get('seller_sales_completed'),
                'Seller Username': item_data.get('seller_username'),
                'Seller Location': item_data.get('seller_location'),
                'Color': item_data.get('color'),
                'Seller Address': item_data.get('seller_address'),
                'Payment Methods': item_data.get('payment_methods'),
                'Description': item_data.get('description')
            }
            all_items.append(item_info)

# Escribo los resultados en un archivo CSV
with open('testMeLi.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Search Term', 'Title', 'Price', 'Condition', 'Sold Quantity', 'Location', 'Discount', 'Seller Rating', 'Seller Sales Completed', 'Seller Username', 'Seller Location', 'Color', 'Seller Address', 'Payment Methods', 'Description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for item in all_items:
        writer.writerow(item)

print("Proceso completado. Los resultados se han guardado en 'testMeLi.csv'.")
```




### Análisis Exploratorio de Datos

Después de obtener y procesar los datos de la API de MercadoLibre, realicé un análisis exploratorio para comprender mejor la distribución y características de los productos obtenidos. A continuación, describo algunos de los análisis realizados y muestro el código generado:

####Análisis de Precios Promedio por Marca:

Se calculó el precio promedio de los productos para cada marca de celular, y grafiqué los precios promedio por marca en un gráfico de barras para visualizar las diferencias.

Se examinó la distribución de precios de los productos para cada marca de celular, con la función describe() para obtener estadísticas descriptivas de los precios por marca, como la media, la desviación estándar, el mínimo, el máximo, etc, y se Graficó la distribución de precios por marca en histogramas superpuestos para comparar visualmente cómo varían los precios entre las marcas.

Se exploró la distribución de precios por marca utilizando un diagrama de caja y bigotes (boxplot), el cual permite identificar la mediana, los cuartiles y los valores atípicos en la distribución de precios para cada marca de celular.

Aquí se muestra el Diagrama diagrama de caja y bigotes (boxplot):

![boxplot](https://github.com/Arzeus17/challenge/blob/main/img/boxPlotPrecioxMarca.png)

Posteriormente se realizó un análisis del estado del producto se enfocado en examinar la proporción de teléfonos nuevos y usados para las cinco marcas principales de celulares. Se creó un gráfico de barras apiladas 2D para mostrar la proporción de celulares nuevos y usados por marca.

Aquí se muestra el gráfico de barras apiladas 2D:

![barras](https://github.com/Arzeus17/challenge/blob/main/img/barraApiladaPrecioxMarca.jpg)

Entre otros analisis, mas para determinar la robustes de los datos obtenidos mediante la API, y el tratamiento adecuado para el analisis final de los mismos.

```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#Cargo el archivo CSV
df = pd.read_csv("testMeLi.csv")

#Análisis de Precios
#Elimino filas con valores faltantes en la columna 'Price'
df = df.dropna(subset=['Price'])

#Convertir la columna 'Price' a tipo numerico
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Calcular el precio promedio por marca
precios_promedio_por_marca = df.groupby('Search Term')['Price'].mean()

# Calcular la distribución de precios por marca
distribucion_precios_por_marca = df.groupby('Search Term')['Price'].describe()

# Imprimir los precios promedio por marca
print("Precios promedio por marca:")
print(precios_promedio_por_marca)

# Imprimir la distribución de precios por marca
print("\nDistribución de precios por marca:")
print(distribucion_precios_por_marca)

# Grafico los precios promedio por marca
precios_promedio_por_marca.plot(kind='bar', color='skyblue')
plt.title('Precios promedio por marca de teléfonos')
plt.xlabel('Marca')
plt.ylabel('Precio promedio (ARS)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Grafico la distribución de precios por marca
for marca, datos in distribucion_precios_por_marca.iterrows():
    plt.hist(df[df['Search Term'] == marca]['Price'], bins=20, alpha=0.5, label=marca)

plt.title('Distribución de precios por marca de teléfonos')
plt.xlabel('Precio (ARS)')
plt.ylabel('Frecuencia')
plt.legend()
plt.tight_layout()
plt.show()

# Grafico un diagrama de caja y bigotes (boxplot)
plt.figure(figsize=(10, 6))
sns.boxplot(x='Search Term', y='Price', data=df)
plt.title('Distribución de Precios por Marca de Dispositivos de Streaming')
plt.xlabel('Marca')
plt.ylabel('Precio')
plt.xticks(rotation=45)  # Rotar las etiquetas del eje x para una mejor visualización
plt.grid(True)
plt.show()


# Análisis del Estado del Producto
# Eliminar filas con valores faltantes en la columna 'Condition'
df = df.dropna(subset=['Condition'])

# Obtener las cinco marcas principales
marcas_top_5 = df['Search Term'].value_counts().index[:5]

# Crear un DataFrame con las cinco marcas
df_marcas = pd.DataFrame(index=marcas_top_5)

# Filtrar el DataFrame original para incluir solo las marcas con celulares usados
df_usados = df[df['Search Term'].isin(marcas_top_5)]

# Calcular el conteo de teléfonos nuevos y usados por marca
conteo_por_marca = df_usados.groupby(['Search Term', 'Condition']).size().unstack()
conteo_por_marca.fillna(0, inplace=True)

# Rellenar las filas con marcas que no tienen celulares usados con ceros
conteo_por_marca = conteo_por_marca.reindex(index=marcas_top_5, fill_value=0)

# Normalizar los valores de cada fila para que sumen 1
conteo_por_marca_norm = conteo_por_marca.div(conteo_por_marca.sum(axis=1), axis=0)

# Crear un gráfico de barras apiladas 2D
conteo_por_marca_norm.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Proporción de teléfonos nuevos y usados por marca')
plt.xlabel('Marca')
plt.ylabel('Proporción')
plt.xticks(rotation=45)
plt.legend(title='Estado del Producto')
plt.tight_layout()
plt.show()


# Análisis de las Ventas del Producto
# Filtrar el DataFrame para incluir solo las columnas de interés (marca y cantidad vendida)
df = df[['Search Term', 'Sold Quantity']]

# Eliminar filas con valores NaN en cualquier columna
df = df.dropna()

# Agrupar los datos por marca y sumar la cantidad vendida para cada una
cantidad_vendida_por_marca = df.groupby('Search Term')['Sold Quantity'].sum()

# Ordenar las marcas por la cantidad vendida de forma descendente
cantidad_vendida_por_marca = cantidad_vendida_por_marca.sort_values(ascending=False)

# Verificar si hay datos para graficar
if not cantidad_vendida_por_marca.empty:
    # Crear un gráfico de torta
    plt.figure(figsize=(8, 6))
    plt.pie(cantidad_vendida_por_marca, labels=cantidad_vendida_por_marca.index, autopct='%1.1f%%', startangle=140)
    plt.title('Proporción de unidades vendidas por marca de teléfonos')
    plt.axis('equal')  # Asegura que el gráfico de torta sea circular
    plt.show()
else:
    print("No hay suficientes datos para graficar.")
```

### Creación de un Dashboard Interactivo en Power BI

Después de completar los análisis exploratorios,  utilicé el archivo de datos CSV, generado a partir del consumo de la API de MercadoLibre.com.ar, y procedí a utilizar Power BI para crear un dashboard interactivo. Este proceso implicó las siguientes etapas:

1. **Preparación y Limpieza de la Base de Datos:**
   - Cargué el archivo de datos CSV en Power BI.
   - Realicé la limpieza de datos, incluyendo la eliminación de valores nulos o duplicados, la corrección de formatos de datos y la manipulación de columnas según fuera necesario.

2. **Elaboración de Gráficos Interactivos:**
   - Utilicé las diversas herramientas y funcionalidades de visualización de Power BI para crear gráficos interactivos que mostraran posibles tendencias y patrones en los datos.
   - Los tipos de gráficos utilizados incluyeron gráficos de barras, mapas, gráficos de torta, entre otros, según la naturaleza de los datos y los insights que se buscaban obtener.

3. **Generación de Insights:**
   - Analicé los gráficos generados para identificar tendencias, correlaciones o anomalías en los datos.
   - Utilicé las funcionalidades de interacción de Power BI, como filtros y segmentaciones, para explorar diferentes aspectos de los datos y profundizar en el análisis.

4. **Creación del Dashboard:**
   - Organicé los gráficos individuales en un dashboard cohesivo y fácil de navegar.
   - Añadí elementos adicionales, como títulos, descripciones y anotaciones, para mejorar la comprensión y la presentación de los insights obtenidos.

El dashboard interactivo en Power BI proporciona una visualización dinámica y fácilmente comprensible de los datos obtenidos a partir de la API de MercadoLibre.com.ar, lo que facilita la identificación de tendencias y la toma de decisiones informadas.

Aquí se muestra una imagen del Dashboard, con los graficos seleccionados:

![Dashboard](https://github.com/Arzeus17/challenge/blob/main/img/Dashboard.png)

### Creación del Repositorio en GitHub

Finalmente, con el objetivo de organizar y compartir todos los archivos relacionados con este dChallenger, he creado un repositorio en mi perfil de GitHub. Este repositorio contiene todos los archivos relevantes, incluyendo el archivo CSV generado a partir del consumo de la API de MercadoLibre.com.ar, los scripts SQL utilizados para la creación de tablas y consultas, el archivo de Power BI con el dashboard interactivo, y este informe generado en Jupyter Notebook con Markdown.

El repositorio en GitHub proporciona un espacio centralizado para almacenar y gestionar todos los activos del proyecto, lo que facilita su acceso y colaboración con otros interesados.

Para acceder al repositorio y revisar los archivos, puede hacer clic en el siguiente enlace: [Repositorio en GitHub](https://github.com/Arzeus17/challenge).
